In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import importlib
from torch.utils.data import DataLoader

from config import Blip2Config
from dataset import Blip2Dataset
from tokenizer import Blip2Tokenizer

In [27]:
import config, dataset, tokenizer
importlib.reload(config)
importlib.reload(dataset)
importlib.reload(tokenizer)

from config import Blip2Config
from dataset import Blip2Dataset
from tokenizer import Blip2Tokenizer

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [28]:
config = Blip2Config()

In [ ]:
blip2_tokenizer = Blip2Tokenizer(config, tokenizer)
train_dataset = Blip2Dataset(config, split="train", tokenizer=blip2_tokenizer.tokenize_text)

Skipping non-image file: image


In [29]:
train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
